In [ ]:
#Keep in mind this is by default set to sample every 4ms 
#To change it you have to change the rate in the ESP32code 
#Working on a way to set sample rate in code 

# Imports
import socket, time

ESP_IP   = "10.0.0.40" # set to ESP32's IP
ESP_PORT = 9000 # must match your sketch
RETRY_SEC = 1.5 # wait before reconnecting

# >>> pick multiple columns to pull from SD CSV/TSV
TARGET_COLS = ["Oz", "O1", "O2"]   # change as needed (names from header)
TARGET_IDXS = None                 # or e.g. [7, 8, 9] (0-based) instead of names

In [ ]:
class _ParserMulti:
    """
    Modes:
      - 'twocol': lines like 't_ms,value' -> yields (t_ms, {'value': float})
      - 'csv'   : multi-column CSV/TSV with header; extracts TARGET_COLS/TARGET_IDXS
                  and converts first 'Time*' column (seconds) -> t_ms (int)
    """
    def __init__(self):
        self.mode = None
        self.delim = None
        self.header = None
        self.time_idx = 0
        self.col_idxs = None
        self.col_names = None

    def reset(self):
        self.__init__()

    def _detect_delim(self, line: bytes):
        if b'\t' in line and b',' not in line: return b'\t'
        if b',' in line and b'\t' not in line: return b','
        return b','

    def _map_columns(self):
        if TARGET_IDXS is not None:
            self.col_idxs = list(TARGET_IDXS)
            # If no names supplied, synthesize names from header if available
            self.col_names = [self.header[i] if self.header and i < len(self.header) else f"col{i}" for i in self.col_idxs]
        else:
            want = [w.lower() for w in TARGET_COLS]
            names_lower = [h.lower() for h in self.header]
            idxs = []
            for w in want:
                try:
                    idxs.append(names_lower.index(w))
                except ValueError:
                    idxs.append(None)  # keep placeholder; we'll skip missing
            self.col_idxs = idxs
            self.col_names = TARGET_COLS

    def handle(self, line: bytes):
        if not line or line.startswith(b"#"):
            return None

        # Try simple two-column 't_ms,value'
        if self.mode in (None, "twocol"):
            sep = line.find(b",")
            if sep != -1 and line.find(b",", sep+1) == -1:
                try:
                    t_ms = int(line[:sep])
                    val  = float(line[sep+1:])
                    self.mode = "twocol"
                    return (t_ms, {"value": val})
                except Exception:
                    pass  # fall through to CSV

        # CSV/TSV flow
        if self.mode is None:
            self.delim = self._detect_delim(line)
            # treat first text line as header
            if any(65 <= c <= 90 or 97 <= c <= 122 for c in line):
                self.header = [tok.strip() for tok in line.decode("utf-8", "ignore").strip().split(self.delim.decode())]
                self.mode = "csv"
                # choose time col: first that starts with "time"
                tids = [i for i, h in enumerate(self.header) if h.lower().startswith("time")]
                if tids: self.time_idx = tids[0]
                self._map_columns()
                return None
            else:
                # no obvious header, still try csv with detected delim
                self.mode = "csv"
                self.delim = self._detect_delim(line)
                # without header we must rely on TARGET_IDXS
                self.header = None
                self.time_idx = 0
                self._map_columns()

        # Parse CSV/TSV data line -> (t_ms, {col: val, ...})
        try:
            parts = [p.strip() for p in line.decode("utf-8", "ignore").strip().split(self.delim.decode())]
            if len(parts) < 2:
                return None
            # time in seconds -> t_ms
            t_s = float(parts[self.time_idx])
            t_ms = int(round(t_s * 1000.0))
            out = {}
            for name, idx in zip(self.col_names, self.col_idxs):
                if idx is None or idx >= len(parts):
                    continue
                try:
                    out[name] = float(parts[idx])
                except Exception:
                    # non-numeric or missing, skip
                    pass
            if not out:
                return None
            return (t_ms, out)
        except Exception:
            return None

def stream_samples(host: str, port: int):
    buf = bytearray()
    parser = _ParserMulti()
    while True:
        s = None
        try:
            s = socket.create_connection((host, port), timeout=5)
            try:  s.setsockopt(socket.SOL_SOCKET, socket.SO_RCVBUF, 256 * 1024)
            except Exception: pass
            try:  s.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
            except Exception: pass
            try:  s.setsockopt(socket.SOL_SOCKET, socket.SO_KEEPALIVE, 1)
            except Exception: pass
            s.settimeout(None)
            print(f"[connected] {host}:{port}")
            buf.clear(); parser.reset()

            while True:
                chunk = s.recv(8192)
                if not chunk:
                    raise ConnectionError("server closed connection")
                buf.extend(chunk)
                nl = buf.find(b"\n")
                while nl != -1:
                    line = buf[:nl].strip()
                    del buf[:nl+1]
                    parsed = parser.handle(line)
                    if parsed is not None:
                        yield parsed
                    nl = buf.find(b"\n")

        except KeyboardInterrupt:
            try:
                if s: s.shutdown(socket.SHUT_RDWR)
            except: pass
            if s: s.close()
            print("\n[bye]"); raise
        except Exception as e:
            print(f"[reconnect] {e}; retrying in {RETRY_SEC}s…")
            try:
                if s: s.shutdown(socket.SHUT_RDWR)
            except: pass
            if s: s.close()
            time.sleep(RETRY_SEC)


In [ ]:
if __name__ == "__main__":
    try:
        for t_ms, vals in stream_samples(ESP_IP, ESP_PORT):
            # vals is a dict, e.g. {'Oz': 0.00123, 'O1': -0.00456, 'O2': 0.00078}
            print(t_ms, vals)
    except KeyboardInterrupt:
        pass
